# F16 Low Level Controller analysis

In this example, we will show how to analyze a low-level-controller.

First, we need to import our model, and a parallel runner

In [ ]:
from run_parallel import run_workgroup
import csaf.config as cconf
import csaf.system as csys

model_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_llc_analyze_config.toml")


This model contains a signal generator, configured to generate a step command. The signal generator is shown below:

In [ ]:
!cat /csaf-system/components/signal_generator.toml

We will also display the system topology, to show the difference between f16-simple and this system:


In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
model_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=150"])

# display written file to notebook
Image(plot_fname)

Next step, we need to prepare initial conditions for our run. For this particular case, we will change only two variables at a time, so we can display the results in a 2D plot. However, you can generate the initial conditions in many different ways.

In [ ]:
# We want to see step response to Nz (z-body accel)
# Varying airspeed and alpha is a good start
initial = [ 540.0, # vt
            0.037027160081059704, # alpha [rad]
            0.0, # beta[rad]
            0.0, # roll [rad]
            0.0, # pitch [rad]
            0.0, # yaw [rad]
            0.0, # p [rad/s]
            0.0, # q [rad/s]
            0.0, # r [rad/s]
            0.0, # pn [m]
            0.0, # pe [e]
            4800.0, # h [ft]
            70.0] # pow

import numpy as np

NUM_SAMPLES=10
delta_vt = np.linspace(-100,100,NUM_SAMPLES) # +- 100gt/s
delta_alpha = np.linspace(-0.3,0.3,NUM_SAMPLES) # +- 0.3rad

# Generate x0s
x0s = []
for d_vt in delta_vt:
    for d_alpha in delta_alpha:
        x0 = initial[:]
        x0[0] += d_vt
        x0[1] += d_alpha
        x0s.append(x0)

# Now create a list of distionaries containing the initial states for F16
states = [{"plant" : np.asarray(x)} for x in x0s]


Now we have 100 different initial conditions. Let's use our parallel runner to efficiently run 100 simulations!

In [ ]:
import run_parallel
n_tasks = len(states)
tspan = (0.0, 10.0) # 10 seconds is enough to get step response
runs = run_workgroup(n_tasks, model_conf, states, tspan)

## Manual analysis

Now we have 100 simulation outputs. Lets examine the response for a step in `Nz` reference.

In [ ]:
# Nz index is 0
nz = [np.array(r["plant"]["outputs"])[:, 0] for r in runs if not isinstance(r, Exception)]
# Nz_ref index is 0
nz_ref = [np.array(r["autopilot"]["outputs"])[:, 0] for r in runs if not isinstance(r, Exception)]
# Fetch time as well
# Because the autopilot runs at different frequency than the plant, we need separate time trace
times_nz = [r["plant"]["times"] for r in runs if not isinstance(r, Exception)]
times_nz_ref = [r["autopilot"]["times"] for r in runs if not isinstance(r, Exception)]

In [ ]:
# Lets plot the results (this might take a while)
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 3 * len(nz)), nrows=len(nz), sharex=True)
for idx in range(len(nz)):
    ax[idx].plot(times_nz[idx], nz[idx])
    ax[idx].plot(times_nz_ref[idx], nz_ref[idx])
    ax[idx].set_ylabel(f"Run {idx}")
    ax[idx].grid(True)
ax[-1].set_xlabel("Time (s)")
ax[0].set_title("Simulation Workgroup Runs")
plt.show()

We can inspect all 100 runs and see how closely the reference was followed. Given the relatively large range of initial conditions, some runs provide a very clean response, while some show large initial `Nz`and large overshoot.

You can modify the initial conditions, as well as the time and amplitude of the step signal to get more meaningful results.

The next question is - can we automatically evaluate the simulation results? The answer is yes!

## More automated analysis

First, define a function that returns the max difference between the reference signal and the response. Note that this is a very simple function meant only as an example. To gain better insight into your system, you should define a more complex function.

In [ ]:
def calculate_simple_difference(ref_time, reference, res_time, response):
    # Assume both arguments are lists
    # Cut off first 1 second of simulation, because that is before the step occured
    start_time = 1.0
    _, ref_idx = min((val, idx) for (idx, val) in enumerate(ref_time) if val > start_time)
    _, res_idx = min((val, idx) for (idx, val) in enumerate(res_time) if val > start_time)
    ref = reference[ref_idx:]
    res = response[res_idx:]
    return max(res) - max(ref)


In [ ]:
# Now calculate the difference for each simulation run
diffs = []
for idx in range(len(runs)):
    delta = calculate_simple_difference(times_nz_ref[idx],nz_ref[idx],times_nz[idx],nz[idx])
    diffs.append(delta)

In [ ]:
# Max "overshoot" occured at last run (99), over 25%
val, idx = max((val, idx) for (idx, val) in enumerate(diffs))
print(val)
print(idx)

In [ ]:
# Min "overshoot" occured at the secind run (1), -3.5% below the reference
val, idx = min((val, idx) for (idx, val) in enumerate(diffs))
print(val)
print(idx)

This seem to indicate that the extreme initial conditions affect the response, which is expected. Lets now plot the results.

In [ ]:
# Import libraries 
from mpl_toolkits import mplot3d 
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib import cm

# Creating dataset
x = []
y = []
for x0 in x0s:
    x.append(x0[0])
    y.append(x0[1])

x = np.asarray(x).reshape((10,10))
y = np.asarray(y).reshape((10,10))
z = np.asarray(diffs).reshape((10,10)) 

fig = plt.figure(figsize =(14, 9)) 
ax = plt.axes(projection ='3d') 

ax.set_xlabel('vt[ft/s]', fontsize=12)
ax.set_ylabel('alpha[rad]', fontsize=12)
ax.set_zlabel('overshoot[%]', fontsize=12)
ax.xaxis.set_rotate_label(True)
ax.yaxis.set_rotate_label(True)

surf = ax.plot_surface(x, y, z,cmap=cm.coolwarm)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

# show plot 
plt.show()

We can see that higher overshoot corresponds to higher alpha, and in lesser extend to higher airspeed.

Finally, if we are only interested in how many simulations do not exceed 10% overshoot, we can simply do:

In [ ]:
len([x for x in diffs if x < 0.1])

## Summary
We demonstrated how to run multiple simulations in parallel, with different initail conditions, and how to analyzer the results. The analysis can be tailer to your needs, the parallel runners will help you with efficient simulation.  